In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('stackoverflow_reduced.csv')

In [3]:
data.head()

,post,tags
0,identifying server timeout quickly in iphone ...,iphone
1,distance between 2 or more drop pins i was do...,iphone
2,what are all the restrictions by apple for iph...,iphone
3,not able to clicked on item i have facing ver...,iphone
4,displaying addressbook data acording to a cert...,iphone


In [4]:
print("Input data has {} rows and {} columns".format(len(data), len(data.columns)))

Input data has 3000 rows and 2 columns


In [5]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
from bs4 import BeautifulSoup

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

In [6]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [7]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text, encoding='latin-1', ngram_range=(1, 2))
X_tfidf = tfidf_vect.fit_transform(data['post'])
X_tfidf_feat = pd.DataFrame(X_tfidf.toarray())
X_tfidf_feat.head()

,0,1,2,3,4,5,6,7,8,9,...,31004,31005,31006,31007,31008,31009,31010,31011,31012,31013
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.082396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
import time

In [9]:
sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
k_fold = KFold(n_splits=10)
cross_val_score(sgd, X_tfidf_feat, data['tags'], cv=k_fold, scoring='accuracy', n_jobs=-1)


array([0.98      , 0.95333333, 0.94666667, 0.95333333, 0.94333333,
       0.95666667, 0.98666667, 0.97333333, 0.96666667, 0.96      ])